In [38]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim as ssim
from random import shuffle
import math
from keras.models import model_from_json, load_model
from keras.layers import *
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
import pickle

from utilities import *
%matplotlib inline


In [39]:
def readcifar(path) : 
    with open(path, 'rb') as f:
        train_set= pickle.load(f)
    return train_set["data"]

In [40]:
raw_train_set1 = readcifar('data/cifar-10-batches-py/data_batch_1')
raw_train_set2 = readcifar('data/cifar-10-batches-py/data_batch_2')
raw_train_set3 = readcifar('data/cifar-10-batches-py/data_batch_3')
raw_train_set4 = readcifar('data/cifar-10-batches-py/data_batch_4')
raw_train_set5 = readcifar('data/cifar-10-batches-py/data_batch_5')

In [41]:
type(raw_train_set1)

numpy.ndarray

In [42]:
raw_train_set = np.concatenate((raw_train_set1,raw_train_set2,raw_train_set3,raw_train_set4,raw_train_set5))

In [43]:
raw_train_set.shape

(50000, 3072)

In [44]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [45]:
def preprocess(images) :
    images = images.reshape(images.shape[0], 3, 32, 32).transpose(0,2,3,1).astype("uint8")
    images = rgb2gray(images)
    images = np.reshape(images, (images.shape[0],32,32,1))
    return images

In [46]:
train_set = preprocess(raw_train_set)

In [47]:
train_set.shape

(50000, 32, 32, 1)

In [48]:
for i in range(100):
    cv2.imwrite("./data/cifar-bw/image"+str(i)+".png",train_set[i])

In [49]:
#Adding noise to images
def add_noise(images):
    #x_train = np.reshape(x_train, (len(x_train), 64*64))  # adapt this if using `channels_first` image data format
    #x_test = np.reshape(x_test, (len(x_test), 64*64))  # adapt this if using `channels_first` image data format
    batch = images.shape[0]//4;
    noise1 = gaussian_noise(images[0:batch],0,25,1)
    noise2 = gaussian_noise(images[batch:2*batch],0,30,1)
    noise3 = gaussian_noise(images[2*batch:3*batch],0,35,1)
    noise4 = gaussian_noise(images[3*batch:],0,40,1)
    
    noisy_set = []
    for data in [noise1,noise2,noise3,noise4]:
        for image in data:
            noisy_set.append(image)
    
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def pair_shuffle(images,noisy_set):
    image_pair = []
    for i in range(images.shape[0]):
        image_pair.append((images[i],noisy_set[i]))
    shuffle(image_pair)
    
    ground_truth=[]
    noisy_images = []
    for i in range(images.shape[0]):
        ground_truth.append(image_pair[i][0])
        noisy_images.append(image_pair[i][1])
    return np.array(ground_truth), np.array(noisy_images)

In [51]:
noisy_set = add_noise(train_set)
for i in range(100):
    cv2.imwrite("./data/noisy-cifar-bw/image"+str(i)+".png",noisy_set[i])

In [52]:
#Shuffling and adding noise to the dataset
ground_truth,noisy_images = pair_shuffle(train_set,noisy_set)

In [97]:
#Split into training and cross validation and normalizing
train_size = int(ground_truth.shape[0]*0.8)
x_train = ground_truth[0:train_size]/255.
x_train_noisy = noisy_images[0:train_size]/255.
x_test = ground_truth[train_size:]/255.
x_test_noisy = noisy_images[train_size:]/255.
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(40000, 32, 32, 1)
(10000, 32, 32, 1)


In [54]:
lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('./models/simple_cnn_autoencoder.csv')
early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/simple_cnn_autoencoder.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [58]:
x_train_noisy[0].shape

(32, 32, 1)

In [66]:
#Defining the model

def get_simple_cnn_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=(x_train_noisy[0].shape))  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='sgd', loss='mean_squared_error',metrics = ['accuracy'])

    print (autoencoder.summary())
    return autoencoder

In [67]:
#Training the model
autoencoder = get_simple_cnn_autoencoder_model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 32, 32, 64)        640       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
up_sampling2d_13 (UpSampling (None, 16, 16, 64)        0         
__________

In [69]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 96s - loss: 0.0296 - acc: 0.0011 - val_loss: 0.0160 - val_acc: 0.0011
Epoch 2/50
40000/40000 [==============================] - 96s - loss: 0.0148 - acc: 0.0011 - val_loss: 0.0138 - val_acc: 0.0011
Epoch 3/50
40000/40000 [==============================] - 98s - loss: 0.0131 - acc: 0.0011 - val_loss: 0.0126 - val_acc: 0.0011
Epoch 4/50
40000/40000 [==============================] - 95s - loss: 0.0121 - acc: 0.0011 - val_loss: 0.0117 - val_acc: 0.0011
Epoch 5/50
40000/40000 [==============================] - 95s - loss: 0.0114 - acc: 0.0011 - val_loss: 0.0111 - val_acc: 0.0011
Epoch 6/50
40000/40000 [==============================] - 95s - loss: 0.0108 - acc: 0.0011 - val_loss: 0.0106 - val_acc: 0.0011
Epoch 7/50
40000/40000 [==============================] - 95s - loss: 0.0104 - acc: 0.0012 - val_loss: 0.0102 - val_acc: 0.0011
Epoch 8/50
40000/40000 [==============================

40000/40000 [==============================] - 95s - loss: 0.0077 - acc: 0.0012 - val_loss: 0.0077 - val_acc: 0.0011
Epoch 27/50
40000/40000 [==============================] - 94s - loss: 0.0077 - acc: 0.0012 - val_loss: 0.0076 - val_acc: 0.0011
Epoch 28/50
40000/40000 [==============================] - 97s - loss: 0.0076 - acc: 0.0012 - val_loss: 0.0076 - val_acc: 0.0011
Epoch 29/50
40000/40000 [==============================] - 93s - loss: 0.0076 - acc: 0.0012 - val_loss: 0.0075 - val_acc: 0.0011
Epoch 30/50
40000/40000 [==============================] - 92s - loss: 0.0075 - acc: 0.0012 - val_loss: 0.0075 - val_acc: 0.0011
Epoch 31/50
40000/40000 [==============================] - 94s - loss: 0.0075 - acc: 0.0012 - val_loss: 0.0074 - val_acc: 0.0011
Epoch 32/50
40000/40000 [==============================] - 96s - loss: 0.0074 - acc: 0.0012 - val_loss: 0.0074 - val_acc: 0.0011
Epoch 33/50
40000/40000 [==============================] - 96s - loss: 0.0074 - acc: 0.0012 - val_loss: 0.007

In [24]:
#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

In [236]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x: x[0]*x[1])([inputLayer,gateFactor])
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return gateFactor,fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(32,32,1))  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        gf1,fg1,c1 = get_gated_connections(0.1,x1)

        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        gf2,fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        gf3,fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        gf4,fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model([input_img,gf1,gf2,gf3,gf4],layers)
        sym_autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy'])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [237]:
sym_autoencoder = get_cnn_dsc_architecture()

Model created
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_13 (InputLayer)            (None, 32, 32, 1)     0                                            
____________________________________________________________________________________________________
conv2d_41 (Conv2D)               (None, 32, 32, 64)    1088        input_13[0][0]                   
____________________________________________________________________________________________________
input_14 (InputLayer)            (1,)                  0                                            
____________________________________________________________________________________________________
lambda_9 (Lambda)                (None, 32, 32, 64)    0           conv2d_41[0][0]                  
                                                                   input_14[0

In [238]:
model_checkpoint1 = ModelCheckpoint('./models/gated_cnn_autoencoder.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [239]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs=100,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 468s - loss: 0.0100 - acc: 0.0012 - val_loss: 0.0054 - val_acc: 0.0011
Epoch 2/100
16340/40000 [===========>..................] - ETA: 257s - loss: 0.0052 - acc: 0.0012

KeyboardInterrupt: 

In [27]:
x_train_noisy.shape

(40000, 32, 32, 1)

In [28]:
save_model_as_json(sym_autoencoder,'CNNDSC_CIFAR')

NameError: name 'sym_autoencoder' is not defined

In [78]:
#Getting images to test
def get_testing_images(filePath):
    test_images = read_dataset(filePath)
    test_images = np.array(test_images)
    test_images = test_images.astype('float32')/255.
    test_images = np.reshape(test_images, (test_images.shape[0],32,32,1))
    noisy_test_images = add_noise(test_images)
    print (noisy_test_images.shape)
    return (test_images,noisy_test_images)

In [155]:
test_images = readcifar('data/cifar-10-batches-py/test_batch')
test_images = preprocess(test_images)
print(test_images.shape)
noisy_test_images = add_noise(test_images)
noisy_test_images = noisy_test_images/255.

(10000, 32, 32, 1)


In [31]:
#Denoising using CNN symmetric Autoencoders
out_sym_autoencoder = sym_autoencoder.predict(noisy_test_images,verbose=1)
print (out_sym_autoencoder.shape)

NameError: name 'sym_autoencoder' is not defined

In [118]:
#Denoising using Autoencoders
out_autoencoder = autoencoder.predict(noisy_test_images,verbose=1).astype('float64')
print (out_autoencoder.shape)

 9984/10000 [============================>.] - ETA: 0s(10000, 32, 32, 1)


In [117]:
test_images.dtype

dtype('float64')

In [119]:
for i in range(100):
    cv2.imwrite("./data/denoise-cifar-bw/"+str(i)+".png",out_autoencoder[i]*255.)

In [57]:
"""
display_images(test_images,10)
display_images(noisy_test_images,10)
display_images(out,10)
"""

'\ndisplay_images(test_images,10)\ndisplay_images(noisy_test_images,10)\ndisplay_images(out,10)\n'

In [132]:
from sklearn.metrics import mean_squared_error
from skimage.measure import compare_psnr,compare_mse
def get_psnr(imageA,imageB):
    maxI = 1.0
    try:
        return 20*math.log10(maxI) - 10*math.log10(compare_mse(imageA.flatten(),imageB.flatten()))
    except:
        return 20*math.log10(maxI)

def get_psnr_result(x_test, out):
    psnr_sum = 0
    for i in range(out.shape[0]):
        psnr_sum += compare_psnr(x_test[i].reshape(32,32,1),out[i].reshape(32,32,1),data_range=255)
        
    return 1.0*psnr_sum/out.shape[0];

def get_ssim_result(originalSet,noisySet):
    ssim_sum = 0
    originalSet = originalSet.reshape(originalSet.shape[0],32,32,1)
    noisySet = noisySet.reshape(noisySet.shape[0],32,32,1)
    for i in range(originalSet.shape[0]):
        ssim_sum += ssim(originalSet[i], noisySet[i],data_range=originalSet[i].max() - noisySet[i].min(), multichannel=True)
    return 1.0*ssim_sum/originalSet.shape[0]

In [165]:
def bm3d_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32)
    denoised = []
    count = 1
    for i in range(noisy_image.shape[0]):
        Basic_img = bm3d.BM3D_1st_step(noisy_image[i])
        Final_img = bm3d.BM3D_2nd_step(Basic_img, noisy_image[i])
        denoised.append(Final_img)
        if (count%10 == 0):
            print (str(count)+ "images denoised")
        count+=1
        
    return np.array(denoised)

def nlm_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,1)
    denoised = []
    count = 1
    
    for image in noisy_image:
        denoised_image = denoise_nl_means(image, 7, 11, 0.5,multichannel = False)
        denoised.append(denoised_image)
        if(count%100 == 0) :
            print(str(count)+" images denoised")
        count+=1
    return np.array(denoised)

In [134]:
print (test_images.shape,out_autoencoder.shape)
print (compare_mse(test_images[0].flatten(),(out_autoencoder[0]*255.).flatten()))
get_psnr_result(out_autoencoder*255.,test_images)

((10000, 32, 32, 1), (10000, 32, 32, 1))
458.092258419


22.066576979897317

In [39]:
print (test_images.shape,out_sym_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_sym_autoencoder[0].flatten()))
get_psnr_result(out_sym_autoencoder,test_images)

NameError: name 'out_sym_autoencoder' is not defined

In [39]:
import bm3d

In [40]:
noisy_test_images.shape
bm3d_out = bm3d_denoise(noisy_test_images)

error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/core/src/dxt.cpp:2330: error: (-213) Odd-size DCT's are not implemented in function dct


In [ ]:
bm3d_out_norm = bm3d_out.astype('float64')/255.0

In [ ]:
get_psnr_result(bm3d_out_norm,test_images)

In [166]:
print(noisy_test_images.shape)
nlm_out = nlm_denoise(noisy_test_images*255.)
nlm_out = nlm_out.astype('float64')

(10000, 32, 32, 1)


KeyboardInterrupt: 

In [157]:
print nlm_out.shape,test_images.shape
get_psnr_result(nlm_out,test_images)

(10000, 32, 32) (10000, 32, 32, 1)


18.414818545893922

In [136]:
get_ssim_result(test_images,out_autoencoder*255.)

0.69608629344603479

In [66]:
get_ssim_result(test_images,out_sym_autoencoder)

0.718734871276698

In [ ]:
get_ssim_result(test_images,bm3d_out_norm)

In [69]:
get_ssim_result(test_images,nlm_out)

0.49438484841716263

In [137]:
model = load_model("./models/simple_cnn_autoencoder.hdf5")

In [21]:
bwmodel= model.predict(noisy_test_images,verbose=1)

10000/10000 [==============================] - 7s     


In [24]:
print (test_images.shape,bwmodel.shape)
get_psnr_result(bwmodel,test_images)

((10000, 32, 32, 1), (10000, 32, 32, 1))


16.773690910050014

In [138]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append((img,filename))
    return np.array(images)
        

In [139]:
standard_test_images = read_images_from_folder('./data/standard_test_images/gd/')

In [140]:
standard_test_images.shape

(12, 2)

In [141]:
standard_test_images[0]

array([ array([[156, 157, 160, ..., 152, 152, 152],
       [156, 157, 159, ..., 152, 152, 152],
       [158, 157, 156, ..., 152, 152, 152],
       ..., 
       [121, 123, 126, ..., 121, 113, 111],
       [121, 123, 126, ..., 121, 113, 111],
       [121, 123, 126, ..., 121, 113, 111]], dtype=uint8),
       'cameraman.tif'], dtype=object)

In [154]:
std_test_images = []
image_names = []
for img in standard_test_images:
    std_test_images.append(img[0])
    image_names.append(img[1])
std_test_images = np.array(test_images)

In [143]:
noisy_std_test_images = add_noise(test_images)

In [144]:
for index in range(noisy_std_test_images.shape[0]):
    cv2.imwrite('./data/standard_test_images/noise/'+image_names[index],noisy_std_test_images[index])

In [222]:
def denoise_by_patches(noisy_image,model,stride):
    denoise_image = [[0 for col in range(512)] for row in range(512)]
    overlap_mat = [[0 for col in range(512)] for row in range(512)]
    denoise_image = np.array(denoise_image)
    overlap_mat = np.array(overlap_mat)
    norm_image = noisy_image/255.
    cnt = 1
    for row in xrange(0,512 - 31,stride):
        for col in xrange(0,512 - 31,stride):
            #print cnt
            cnt+=1
            patch = norm_image[row:row+32,col:col+32]
            #print patch.shape
            de_patch = [patch]
            de_patch = np.array(de_patch)
            #print row,col,de_patch.shape
            de_patch = np.reshape(de_patch, (1,32,32,1))
            #print de_patch.shape
            de_patch = model.predict(de_patch)
            de_patch = de_patch[0]*255.
            denoise_image[row:row+32 ,col:col+32] += np.reshape(de_patch, (32,32)).astype('int64')
            overlap_mat[row:row+32 ,col:col+32] += 1
    
    for row in xrange(0,512):
        for col in xrange(0,512):
            denoise_image[row][col]/=(overlap_mat[row][col]*1.0)
    
    return denoise_image


In [229]:
def denoise_std_test_images(images,model,stride):
    denoised_images = []
    for index in range(images.shape[0]):
        d_image = denoise_by_patches(images[index],model,stride)
        denoised_images.append(d_image)
        print compare_psnr(d_image.astype('int64'),std_test_images[index].astype('int64'),data_range=255)
        cv2.imwrite('./data/standard_test_images/denoise/'+image_names[index],d_image)
        print "image " + str(index)+ " denoised"

In [231]:
denoise_std_test_images(noisy_std_test_images,model,1)

KeyboardInterrupt: 

In [196]:
nlm_camera = cv2.fastNlMeansDenoising(noisy_std_test_images[0].astype('uint8'),None,25,7 ,21)
cv2.imwrite("nlm.png",nlm_camera)
print compare_psnr(nlm_camera,std_test_images[0])

30.1276904426
